<font size="8">Trace Analysis Examples</font>
<br>
<font size="5">Tasks Latencies</font>
<br>
<hr>

# Import Required Modules

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)

In [2]:
# Generate plots inline
%matplotlib inline

import json
import os

# Support to access the remote target
import devlib
from env import TestEnv

# Support for workload generation
from wlgen import RTA, Ramp

# Support for trace analysis
from trace import Trace
from trace_analysis import TraceAnalysis

# Support for plotting
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import trappy

# Target Configuration

In [3]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'juno',
    "host"        : '192.168.0.1',

    # Folder where all the results will be collected
    "results_dir" : "TraceAnalysis_TaskLatencies",

    # Define devlib modules to load
    "exclude_modules" : [ 'hwmon' ],

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
        "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
        ],
        
         "buffsize" : 100 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'rt-app' ],
    
    # Comment this line to calibrate RTApp in your own platform
    "rtapp-calib" :  {"0": 360, "1": 142, "2": 138, "3": 352, "4": 352, "5": 353},
}

In [4]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False, force_new=True)
target = te.target

02:05:04  INFO    :         Target - Using base path: /home/brejac01/sources/lisa
02:05:04  INFO    :         Target - Loading custom (inline) target configuration
02:05:04  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq']
02:05:04  INFO    :         Target - Connecting linux target:
02:05:04  INFO    :         Target -   username : root
02:05:04  INFO    :         Target -       host : 192.168.0.1
02:05:04  INFO    :         Target -   password : 
02:05:04  INFO    :         Target - Connection settings:
02:05:04  INFO    :         Target -    {'username': 'root', 'host': '192.168.0.1', 'password': ''}


ExceptionPxssh: could not set shell prompt (received: "unset PROMPT_COMMAND\r\nPS1='[PEXPECT]\\$ '\r\nset prompt='[PEXPECT]\\$ '\r\n", expected: '\\[PEXPECT\\][\\$\\#] ').

# Workload Execution

In [ ]:
def experiment(te):

    # Create and RTApp RAMP task
    rtapp = RTA(te.target, 'ramp', calibration=te.calibration())
    rtapp.conf(kind='profile',
               params={
                    'ramp' : Ramp(
                        start_pct =  60,
                        end_pct   =  20,
                        delta_pct =   5,
                        time_s    =   0.5).get()
              })

    # FTrace the execution of this workload
    te.ftrace.start()
    rtapp.run(out_dir=te.res_dir)
    te.ftrace.stop()

    # Collect and keep track of the trace
    trace_file = os.path.join(te.res_dir, 'trace.dat')
    te.ftrace.get_trace(trace_file)
    
    # Collect and keep track of the Kernel Functions performance data
    stats_file = os.path.join(te.res_dir, 'trace.stats')
    te.ftrace.get_stats(stats_file)

    # Dump platform descriptor
    te.platform_dump(te.res_dir)

In [ ]:
experiment(te)

# Parse Trace and Profiling Data

In [ ]:
# Base folder where tests folder are located
res_dir = te.res_dir
logging.info('Content of the output folder %s', res_dir)
!tree {res_dir}

In [ ]:
with open(os.path.join(res_dir, 'platform.json'), 'r') as fh:
    platform = json.load(fh)
#print json.dumps(platform, indent=4)
logging.info('LITTLE cluster max capacity: %d',
             platform['nrg_model']['little']['cpu']['cap_max'])

In [ ]:
trace_file = os.path.join(res_dir, 'trace.dat')
trace = Trace(platform, trace_file, events=my_conf['ftrace']['events'])

# Trace visualization

In [ ]:
trappy.plotter.plot_trace(trace.ftrace)

# Latency DataFrames

In [ ]:
# Report full set of task status informations available from the trace
trace.data_frame.latency_df('ramp').head()

In [ ]:
df = trace.data_frame.trace_event('sched_switch')
df.head()

In [ ]:
# Report WAKEUP events and their duration
trace.data_frame.latency_wakeup_df('ramp').head()

In [ ]:
# Report PREEMPTION events and their duration
trace.data_frame.latency_preemption_df('ramp').head()

## Latency Plots

In [ ]:
# Plot latency events for a specified task
latency_stats_df = trace.analysis.latency.plotLatency('ramp')

In [ ]:
# Plot statistics on task latencies
latency_stats_df.T

In [ ]:
# Plot latency events for a specified task
trace.analysis.latency.plotLatencyBands('ramp')

In [ ]:
# Zoom into a spefific time frame
trace.setXTimeRange(3.445, 3.45)
trace.analysis.latency.plotLatencyBands('ramp')